NOTE: This notebook assumes that you have downloaded the competition data and saved it in `./data/speed-and-structure-train-data` and `./data/speed-and-structure-train-data-extended` directories. The 1_eda.ipynb notebook also must be run before this notebook to generate the `fold_info_all.csv` file.

# Speed and Structure Competition

## Part 2: Synthetic Data Generation

---

We are generating new velocity samples based on the training data. Each new sample is a weighted average of two random original samples. The weight is randomly sampled from a uniform distribution.

$$
V_{new} = w V_1 + (1 - w) V_2
$$
where $w \in [0, 1]$.

This is very loosely inspired by the oversampling method [SMOTE](https://arxiv.org/abs/1106.1813) with k=1. 

For each generated velocity sample, we use the devito library to simulate the seismic data.

The generated data won't be exactly from the same distribution as the training data so we won't add any of them to the validation set. They will only be used for training.

Note that this notebook will generate 10 samples for demonstration purposes. You should generate more data for training.

In [ ]:
import os
from glob import glob
import random

import numpy as np
import matplotlib.pyplot as plt
from examples.seismic import TimeAxis, RickerSource, Receiver, Model 
from devito import TimeFunction, Eq, solve, Operator
from tqdm.auto import tqdm
from scipy.interpolate import interp1d
import pandas as pd

Important Note: I generated 20000 samples for the competition. 

In [ ]:
# I generated 20000 samples for the competition. 
GEN_N = 10 # This is for demonstration. You should generate more data for training.
# GEN_N = 20000
SRC_INDICES = [1, 75, 150, 225, 300]
OUTPUT_DIR = "./data/synth_data"
OUTPUT_FOLD_INFO_DIR = "./"
os.makedirs(OUTPUT_DIR, exist_ok=True)

TRAINING_DATASET_1 = "./data/speed-and-structure-train-data-extended/*"  
TRAINING_DATASET_2 = "./data/speed-and-structure-train-data/*"  
FOLD_INFO_FILE = "./fold_info_all.csv" # fold info for the original and extended dataset 

### Simulation Configurations
This class and the `vel_to_seis` function below is the best set of simulation configurations I found for the given dataset. I followed this thread in the competition forum [here](https://forum.thinkonward.com/t/synthatic-data-generation/2388/4) and the devito documentation [here](https://www.devitoproject.org/examples/seismic/tutorials/01_modelling.html). I wasn't able to run the simulation on a GPU so my implementation here is CPU based and probably suboptimal.

If you don't explicitly set `dt=0.1`, the critical dt will be ~0.1548 and the resulting number of time steps will be 6461 instead of 10001. The generation process will be ~1.5 times faster but the data will have less resolution and obviously a different shape. For my solution, I don't mind the loss of fidelity since I'm already interpolating the data down to 2048 time steps.

In [ ]:
class VelocityModel:
    def __init__(self, t0, tn, 
                 v, 
                 src_ind,
                 f0, rec_num, 
                 origin=(0., 0.), spacing=(1., 1.)):
        self.t0 = t0
        self.tn = tn
        self.origin = origin
        self.spacing = spacing

        self.model = Model(
                vp=v, 
                origin=self.origin, 
                shape=v.shape, 
                spacing=self.spacing,
                space_order=2, 
                nbl=350, 
                bcs="damp", 
                # dt=self.dt, # it will be ~0.1548 if we don't set dt
            # dtype=np.float64
        )
        self.dt = self.model.critical_dt


        self.time_range = TimeAxis(start=t0, stop=tn, step=self.dt)
        # self.grid = Grid(shape=grid_shape, extent=grid_extent)
        self.src = RickerSource(name='src', grid=self.model.grid, f0=f0, time_range=self.time_range)
        self.src.coordinates.data[0, 0] = src_ind - 1
        self.src.coordinates.data[0, -1] = 20.  # Depth is 20m

        self.rec = Receiver(name='rec', grid=self.model.grid, time_range=self.time_range, npoint=rec_num)
        
        # this should be parametric
        self.rec.coordinates.data[:, 0] = np.asarray([i for i in range(0, 301, 10)])
        self.rec.coordinates.data[:, 1] = 10  # Depth is 10m


        self.u = TimeFunction(name="u", grid=self.model.grid, time_order=2, space_order=2)
        self.pde = self.model.m * self.u.dt2 - self.u.laplace + self.model.damp * self.u.dt
        self.stencil = Eq(self.u.forward, solve(self.pde, self.u.forward))

        # Finally we define the source injection and receiver read function to generate the corresponding code
        self.src_term = self.src.inject(field=self.u.forward, expr=self.src * self.dt**2 / self.model.m)

        # Create interpolation expression for receivers
        self.rec_term = self.rec.interpolate(expr=self.u.forward)

        self.op = Operator([self.stencil] + self.src_term + self.rec_term, subs=self.model.spacing_map)

    def get_seis(self):
        self.op(time=self.time_range.num-1, dt=self.model.critical_dt)

        rec_data_sim = np.asarray(self.rec.data)
        # For some reason, removing the last row and padding the first row seems to help
        # remove the last row
        rec_data_sim = rec_data_sim[:-1]
        # pad the first row
        rec_data_sim = np.pad(rec_data_sim, ((1, 0), (0, 0)))

        return rec_data_sim

In [ ]:
def vel_to_seis(target_data, src_ind):
    vel_model = VelocityModel(
        t0=0.,
        tn=1000.,
        v=target_data,
        src_ind=src_ind,
        f0=0.015,  # Source peak frequency is 15Hz (0.015 kHz)
        rec_num=31
    )

    rec_data_sim = vel_model.get_seis()
    return rec_data_sim

In [ ]:
# List to store the names of subfolders (sample IDs)
sample_paths_1 = glob(TRAINING_DATASET_1)
sample_paths_2 = glob(TRAINING_DATASET_2)
sample_paths = sample_paths_1 + sample_paths_2
# extract the name of samples, i.e. sample IDs
sample_ids = [path.split("/")[-1] for path in sample_paths[:4]]

### Synthetic Velocity Generation
This is the function that returns a velocity model as mentioned above.

In [ ]:
print("Number of samples:", len(sample_paths))

def create_new_vp(_sample_paths):
    # pick two random samples
    ind_1 = random.randint(0, len(_sample_paths) - 1)
    ind_2 = random.randint(0, len(_sample_paths) - 1)

    # target velocity model
    target_data_1 = np.load(os.path.join(_sample_paths[ind_1], "vp_model.npy"))
    target_data_2 = np.load(os.path.join(_sample_paths[ind_2], "vp_model.npy"))

    # flip with prob 0.5
    if random.random() > 0.5:
        target_data_1 = np.flip(target_data_1, axis=0)
    if random.random() > 0.5:
        target_data_2 = np.flip(target_data_2, axis=0)

    # weighted average
    w = random.random()
    target_data = w * target_data_1 + (1 - w) * target_data_2

    return target_data

### Synthetic Seismic Data Generation
Here we use the `vel_to_seis` function to generate synthetic seismic data and compare it with the original seismic data.

In [ ]:
ind = 0 # let's pick the first sample
target_data = np.load(os.path.join(sample_paths[ind], "vp_model.npy"))
src_ind = 150 # the source index (The third source)
rec_data = np.load(os.path.join(sample_paths[ind], f"receiver_data_src_{src_ind}.npy"))

In [ ]:
rec_sim = vel_to_seis(target_data, src_ind)
rec_sim.shape

Since the original data has 10001 time steps and the generated data has 6461 time steps, we need to interpolate both to a smaller number of time steps (e.g. 1001) to make them comparable.

In [ ]:
# interpolate rec_sim from (6461, 31) to (1001, 31)
original_rows = rec_sim.shape[0]
target_rows = 1001
x_original = np.linspace(0, 1, original_rows)
x_new = np.linspace(0, 1, target_rows)

# Interpolate along axis 0 (rows), keeping columns unchanged
interpolator = interp1d(x_original, rec_sim, axis=0, kind='linear')
rec_sim_interpolated = interpolator(x_new)

print(rec_sim_interpolated.shape)  # Should print (1001, 31)

### Comparison
We calculate how diffent they are on average and the maximum difference.

In [ ]:
diff = rec_sim_interpolated - rec_data[::10, :] # downsample the original data
print(np.abs(diff).mean())
print(np.abs(diff).max())

### Visualization
We are making sure that they are visually similar as well.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(rec_sim_interpolated, aspect="auto", cmap="jet")
ax[1].imshow(rec_data[::10, :], aspect="auto", cmap="jet")
plt.show()

### Generate Synthetic Data
Finally, we generate the synthetic data. We are making sure that they are not in the validation set by setting the fold to -1. New `dir_id`s are simply numbers starting from 1.

In [ ]:
df_list = []

for i in tqdm(range(GEN_N)):
    dir_id = i+1
    fold = -1 # set fold to -1 so that they will never be in any validation set
    os.makedirs(f"{OUTPUT_DIR}/{dir_id}", exist_ok=True)

    vel_file = f"{OUTPUT_DIR}/{dir_id}/vp_model.npy"
    rec_1 = f"{OUTPUT_DIR}/{dir_id}/receiver_data_src_1.npy"
    rec_75 = f"{OUTPUT_DIR}/{dir_id}/receiver_data_src_75.npy"
    rec_150 = f"{OUTPUT_DIR}/{dir_id}/receiver_data_src_150.npy"
    rec_225 = f"{OUTPUT_DIR}/{dir_id}/receiver_data_src_225.npy"
    rec_300 = f"{OUTPUT_DIR}/{dir_id}/receiver_data_src_300.npy"

    df_list.append({"dir_id": dir_id, "fold": fold, "vel_file": vel_file, "rec_1": rec_1, "rec_75": rec_75, "rec_150": rec_150, "rec_225": rec_225, "rec_300": rec_300})

    # check if file exists
    if os.path.exists(vel_file) and os.path.exists(rec_1) and os.path.exists(rec_75) and os.path.exists(rec_150) and os.path.exists(rec_225) and os.path.exists(rec_300):
        print("File already exists. Skipping...")
        continue

    new_target_data = create_new_vp(sample_paths)
    rec_data_sim = []
    for src_ind in SRC_INDICES:
        rec_data_ind = vel_to_seis(new_target_data, src_ind)
        rec_data_sim.append(rec_data_ind)
    # This loop can be parallelized like so
    # rec_data_sim = Parallel(n_jobs=4)(delayed(vel_to_seis)(new_target_data, src_ind) for src_ind in SRC_INDICES)
    
    # save the data
    np.save(vel_file, new_target_data)
    for i, s in enumerate(SRC_INDICES):
        rec_data = rec_data_sim[i]
        np.save(f"{OUTPUT_DIR}/{dir_id}/receiver_data_src_{s}.npy", rec_data)

df = pd.DataFrame(df_list, columns=["dir_id", "fold", "vel_file", "rec_1", "rec_75", "rec_150", "rec_225", "rec_300"])
df.to_csv(f"{OUTPUT_FOLD_INFO_DIR}/fold_info_synth_data.csv", index=False)

In [ ]:
# Create a new fold_info file including the synthetic data
fold_info_orig = pd.read_csv(FOLD_INFO_FILE)
fold_info_with_synth = pd.concat([fold_info_orig, df], ignore_index=True)
fold_info_with_synth.to_csv(f"{OUTPUT_FOLD_INFO_DIR}/fold_info_all_with_synth.csv", index=False)
fold_info_with_synth.head()